In [1]:
options(warn=-1)

suppressMessages(library(ArchR))
suppressMessages(library(Seurat))
suppressMessages(library(Signac))
suppressMessages(library(harmony))
suppressMessages(library(dplyr))
suppressMessages(library(cowplot))
suppressMessages(library(harmony))
suppressMessages(library(Nebulosa))
suppressMessages(library(ggpubr))
suppressMessages(library(Ipaper))
suppressMessages(library(phateR))
suppressMessages(library(parallel))
suppressMessages(library(ggrepel))
suppressMessages(library(circlize))

In [2]:
set.seed(42)
getwd()

addArchRThreads(threads = parallel::detectCores() - 2)
addArchRGenome("hg38")

[1] "/data/scATA/SingleCellOpenChromatin/local/VisiumHeartRevision/IntegrativeAnalysis/Fibroblast/code"

Setting default number of Parallel threads to 126.

Setting default genome to Hg38.



In [3]:
## subset atac project
proj <- loadArchRProject("../data/snATAC_SCARA5_Myofib", showLogo = FALSE)

Successfully loaded ArchRProject!



In [6]:
## we first plot the activity and expression of the selected TFs
df_tf <- read.csv("../data/snATAC_SCARA5_Myofib/sel_tf_by_expression.csv", row.names = 1)
head(df_tf)

,idx1,idx2,matchname1,matchname2,name1,name2,Correlation,VarAssay1,VarAssay2,TStat,Pval,FDR
,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
5,17,3251,RELA,RELA,z:RELA_17,chr11:RELA,0.1537066,0.8278041,0.5642279,1.539917,0.12680337,0.9308782
6,20,15086,ZNF354C,ZNF354C,z:ZNF354C_20,chr5:ZNF354C,0.1032200,0.6714060,0.6586250,1.027314,0.30680183,0.9533176
7,21,6967,CTCF,CTCF,z:CTCF_21,chr16:CTCF,0.1207020,0.8436019,0.8611900,1.203690,0.23160901,0.9533176
14,37,14546,MEF2C,MEF2C,z:MEF2C_37,chr5:MEF2C,0.2249046,0.8088468,0.6963629,2.284981,0.02446968,0.5334390
18,50,7725,STAT3,STAT3,z:STAT3_50,chr17:STAT3,0.1020400,0.6271722,0.8251866,1.015445,0.31239421,0.9533176
19,53,16717,FOXP2,FOXP2,z:FOXP2_53,chr7:FOXP2,0.1169310,0.7898894,0.8924629,1.165554,0.24662388,0.9533176


In [7]:
## obtain tf activity along the trajectory
trajMM  <- getTrajectory(ArchRProj = proj,
                         name = "SCARA5_Myofib_dm",
                         useMatrix = "MotifMatrix",
                         log2Norm = FALSE,
                         scaleTo = NULL,
                        smoothWindow = 11)

trajMM <- trajMM[!grepl("deviations", rownames(trajMM)), ]

matMM <- plotTrajectoryHeatmap(trajMM, 
                            varCutOff = 0,
                            pal = paletteContinuous(set = "solarExtra"),
                            limits = c(-2, 2),
                              returnMatrix = TRUE)

df_motif_activity <- as.data.frame(matMM[df_tf$name1, ]) 
colnames(df_motif_activity) <- 1:100
df_motif_activity$tf <- rownames(df_motif_activity)

df_motif_activity2 <- tidyr::pivot_longer(df_motif_activity, -tf, names_to = "pseudotime", values_to = "tf_activity")

df_motif_activity2$pseudotime <- as.numeric(df_motif_activity2$pseudotime)
df_motif_activity2$tf <- stringr::str_split_fixed(df_motif_activity2$tf, ":|_", 3)[, 2]

Creating Trajectory Group Matrix..

Smoothing...

ArchR logging to : ArchRLogs/ArchR-plotTrajectoryHeatmap-31ece6af1adcd-Date-2022-02-24_Time-19-42-27.log
If there is an issue, please report to github with logFile!

useSeqnames is NULL or greater than 1 with a Sparse.Assays.Matrix trajectory input.

2022-02-24 19:42:27 : 

force=FALSE thus continuing with subsetting useSeqnames = z

2022-02-24 19:42:27 : 

Preparing Main Heatmap..

'magick' package is suggested to install to give better rasterization.

Set `ht_opt$message = FALSE` to turn off this message.

ArchR logging successful to : ArchRLogs/ArchR-plotTrajectoryHeatmap-31ece6af1adcd-Date-2022-02-24_Time-19-42-27.log



In [8]:
trajGEX <- getTrajectory(ArchRProj = proj,
                         name = "SCARA5_Myofib_dm",
                         useMatrix = "GeneExpressionMatrix",
                         log2Norm = TRUE,
                        smoothWindow = 11)

trajGEX <- trajGEX[df_tf$name2, ]

gene_expression <- plotTrajectoryHeatmap(trajGEX,
                        varCutOff = 0,
                        pal = paletteContinuous(set = "horizonExtra"),
                        limits = c(-2, 2),
                                         returnMatrix = TRUE)

df_gene_expression <- as.data.frame(gene_expression) 
colnames(df_gene_expression) <- 1:100
df_gene_expression$tf <- rownames(df_gene_expression)

df_gene_expression2 <- tidyr::pivot_longer(df_gene_expression, -tf, names_to = "pseudotime", values_to = "tf_expression")

df_gene_expression2$pseudotime <- as.numeric(df_gene_expression2$pseudotime)
df_gene_expression2$tf <- stringr::str_split_fixed(df_gene_expression2$tf, ":", 2)[, 2]

Creating Trajectory Group Matrix..

Smoothing...

ArchR logging to : ArchRLogs/ArchR-plotTrajectoryHeatmap-31ece626facd70-Date-2022-02-24_Time-19-42-43.log
If there is an issue, please report to github with logFile!

Preparing Main Heatmap..

'magick' package is suggested to install to give better rasterization.

Set `ht_opt$message = FALSE` to turn off this message.

ArchR logging successful to : ArchRLogs/ArchR-plotTrajectoryHeatmap-31ece626facd70-Date-2022-02-24_Time-19-42-43.log



In [10]:
if(!dir.exists("../data/snATAC_SCARA5_Myofib/VisGRN_V2/TF")){
    dir.create("../data/snATAC_SCARA5_Myofib/VisGRN_V2/TF", recursive = TRUE)
}

write.csv(df_motif_activity2, "../data/snATAC_SCARA5_Myofib/VisGRN_V2/TF_activity.csv")
write.csv(df_gene_expression2, "../data/snATAC_SCARA5_Myofib/VisGRN_V2/TF_expression.csv")

for(sel_tf in unique(df_motif_activity2$tf)){
        df_plot1 <- subset(df_motif_activity2, tf == sel_tf) %>%
            rename(value = tf_activity) %>%
            mutate(value = scale(value)) %>%
            mutate(data = "Activity")

        
        df_plot2 <- subset(df_gene_expression2, tf == sel_tf) %>%
            rename(value = tf_expression)%>%
            mutate(value = scale(value)) %>%
            mutate(data = "Expression")
        
        df_plot <- rbind(df_plot1, df_plot2)
        
        p <- ggplot(data = df_plot, aes(x = pseudotime, y = value, color = data)) +
        geom_smooth(method = "loess",se = FALSE, formula = y ~ x) +
        theme_cowplot() +
        theme(legend.title = element_blank())
    
         ggsave(glue::glue("../data/snATAC_SCARA5_Myofib/VisGRN_V2/TF/{sel_tf}.pdf"),
                  plot = p, height = 3, width = 5)
        
}

In [11]:
df_tf_gene <- read.csv("../data/snATAC_SCARA5_Myofib/gene_regulatory_network.csv")

#df_tf_gene$tf <- stringr::str_split_fixed(df_tf_gene$tf, ":", 2)[, 2]

head(df_tf_gene)
nrow(df_tf_gene)

,tf,gene,correlation
,<chr>,<chr>,<dbl>
1,KLF2,PCOLCE2,0.9462578
2,KLF4,PCOLCE2,0.9476462
3,SP3,PCOLCE2,0.9405444
4,ATF4,PCOLCE2,0.9121518
5,ELF2,PCOLCE2,0.7645778
6,THAP11,PCOLCE2,0.8967850


[1] 7992

In [12]:
trajGEX <- getTrajectory(ArchRProj = proj,
                         name = "SCARA5_Myofib_dm",
                         useMatrix = "GeneExpressionMatrix",
                         log2Norm = TRUE,
                        smoothWindow = 11)

Creating Trajectory Group Matrix..

Smoothing...



In [14]:
gene_expression <- plotTrajectoryHeatmap(trajGEX,
                                         varCutOff = 0.5,
                                         limits = c(-2, 2),
                                         returnMatrix = TRUE)

rownames(gene_expression) <- stringr::str_split_fixed(rownames(gene_expression), ":", 2)[, 2]

head(gene_expression)

ArchR logging to : ArchRLogs/ArchR-plotTrajectoryHeatmap-31ece626ffa68d-Date-2022-02-24_Time-19-46-38.log
If there is an issue, please report to github with logFile!

Preparing Main Heatmap..

'magick' package is suggested to install to give better rasterization.

Set `ht_opt$message = FALSE` to turn off this message.

ArchR logging successful to : ArchRLogs/ArchR-plotTrajectoryHeatmap-31ece626ffa68d-Date-2022-02-24_Time-19-46-38.log



,T.0_1,T.1_2,T.2_3,T.3_4,T.4_5,T.5_6,T.6_7,T.7_8,T.8_9,T.9_10,⋯,T.90_91,T.91_92,T.92_93,T.93_94,T.94_95,T.95_96,T.96_97,T.97_98,T.98_99,T.99_100
PCOLCE2,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.911464,1.820656,1.891962,⋯,-1.0551072,-1.0310971,-1.0494265,-1.0943677,-1.1267927,-1.1267927,-1.1267927,-1.1267927,-1.1267927,-1.1267927
SCARA5,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.947554,1.896040,⋯,-0.5837856,-0.6152117,-0.7004977,-0.6476685,-0.7507156,-0.7507156,-0.7507156,-0.7507156,-0.7507156,-0.7507156
SDK1,1.782181,1.782181,1.782181,1.782181,1.782181,1.782181,1.692772,1.668100,1.651542,1.647396,⋯,-0.6158370,-0.5637610,-0.6299809,-0.6430957,-0.8230578,-0.8230578,-0.8230578,-0.8230578,-0.8230578,-0.8230578
UAP1,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.974695,1.818394,1.799603,1.859479,⋯,-0.6456419,-0.6696940,-0.6556953,-0.6585852,-0.5859250,-0.5859250,-0.5859250,-0.5859250,-0.5859250,-0.5859250
ZBTB7C,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,⋯,-0.9807516,-0.9769244,-0.9507731,-0.8322024,-0.8336563,-0.8336563,-0.8336563,-0.8336563,-0.8336563,-0.8336563
PLCB1,1.536287,1.536287,1.536287,1.536287,1.536287,1.536287,1.455542,1.289545,1.373181,1.470684,⋯,-0.8993177,-0.9039852,-0.8976588,-0.8828782,-1.0017770,-1.0017770,-1.0017770,-1.0017770,-1.0017770,-1.0017770


In [15]:
df_gene_expression <- as.data.frame(gene_expression) 
colnames(df_gene_expression) <- 1:100
df_gene_expression$gene <- rownames(df_gene_expression)
df_gene_expression2 <- tidyr::pivot_longer(df_gene_expression, -gene, 
                                           names_to = "pseudotime", values_to = "gene_expression")
df_gene_expression2$pseudotime <- as.numeric(df_gene_expression2$pseudotime)

write.csv(df_gene_expression2, "../data/snATAC_SCARA5_Myofib/VisGRN_V2/TF_Target_Gene_expression.csv")

In [16]:
if(!dir.exists(glue::glue("../data/snATAC_SCARA5_Myofib/VisGRN_V2/TF_Target_Expression"))){
        dir.create(glue::glue("../data/snATAC_SCARA5_Myofib/VisGRN_V2/TF_Target_Expression"), 
                   recursive = TRUE)
}

for(name1 in unique(df_tf_gene$gene)){
        df_plot <- subset(df_gene_expression2, gene == name1)

        p <- ggplot(df_plot, aes(x = pseudotime, y = gene_expression)) +
         geom_smooth(method = "loess",se = FALSE, formula = y ~ x) +
        theme_cowplot() + ggtitle(name1)
    
        ggsave(glue::glue("../data/snATAC_SCARA5_Myofib/VisGRN_V2/TF_Target_Expression/{name1}.pdf"),
              plot = p, height = 4, width = 5)

    }